In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

### Entrega 10/11 - Dataset lido
Mínimo esperado: um arquivo do jupyter notebook (.ipynb) com o
código que lê o dataset (que também deve ser enviado no git) e realiza limpeza e manipulações necessárias no dataset.

In [2]:
data = pd.read_csv("wine.csv", ";")
data = pd.get_dummies(data)

print(data.shape)
data.head()

(6497, 14)


,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality,type_Red,type_White
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,0,1
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,0,1
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,0,1
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,0,1
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,0,1


### Entrega 15/11 - Dataset lido e análise exploratória feita
Mínimo esperado: um arquivo do jupyter notebook (.ipynb) com o
código que lê o dataset (que também deve ser enviado no git) e faz uma análise exploratória inicial.

In [3]:
for col in data.columns:
    print('-' * 40 + col + '-' * 40 , end=' - ')
    display(data[col].value_counts())

----------------------------------------fixed.acidity---------------------------------------- - 

6.80     354
6.60     327
6.40     305
7.00     282
6.90     279
        ... 
14.30      1
15.90      1
13.80      1
14.20      1
6.45       1
Name: fixed.acidity, Length: 106, dtype: int64

----------------------------------------volatile.acidity---------------------------------------- - 

0.280    286
0.240    266
0.260    256
0.250    238
0.220    235
        ... 
0.805      1
1.130      1
1.185      1
0.565      1
0.865      1
Name: volatile.acidity, Length: 187, dtype: int64

----------------------------------------citric.acid---------------------------------------- - 

...

6    2836
5    2138
7    1079
4     216
8     193
3      30
9       5
Name: quality, dtype: int64

----------------------------------------type_Red---------------------------------------- - 

0    4898
1    1599
Name: type_Red, dtype: int64

----------------------------------------type_White---------------------------------------- - 

1    4898
0    1599
Name: type_White, dtype: int64

A variável target escolhida é "quality" para descobrir qual é a qualidade do vinho analisado. As outras vão ser consideradas features.

### Preparação dos dados
Usando numpy, é possível separar o target dos features e simplificar o processo do algoritmo de machine learning escolhido.

In [4]:
target = data["quality"]

features = data.drop("quality", axis = 1)

print(target.shape)
print(features.shape)

(6497,)
(6497, 13)


### Separando dos dados em treinamento e teste

In [5]:
trn_features, tst_features, trn_target, tst_target = train_test_split(features, target, test_size = 0.3)

### Treinando o 1° modelo
O primeiro modelo escolhido é o Random Forest

In [6]:
# O modelo vai ser iniciado com 1000 árvores
rf = RandomForestClassifier(oob_score = True, n_estimators = 1000, max_features = 5, n_jobs=-1, criterion = "entropy")
rf.fit(trn_features, trn_target);

### Testando o 1° modelo

In [7]:
# Probabilidade das previsões do modelo
predictions_prob_rf = rf.predict_proba(tst_features)
predictions_rf = rf.predict(tst_features)

error_rf = 1 - rf.score(tst_features, tst_target)

# Erro total de classificação
print('Prediction prob:',round(np.mean(predictions_prob_rf),4)*100, "%")
print('Mean Absolute Error:', round(error_rf,4)*100, "%")

# Matriz de confusão
pd.crosstab(tst_target, predictions_rf, rownames=['Actual Preference'], colnames=['Predicted Preference'])

Prediction prob: 14.29 %
Mean Absolute Error: 31.85 %


Predicted Preference,4,5,6,7,8
Actual Preference,,,,,
3,0,6,3,0,0
4,9,25,14,1,0
5,2,481,181,2,0
6,2,140,663,48,0
7,0,5,144,160,2
8,0,1,26,17,16
9,0,0,1,1,0


### Treinando o 2° modelo
O segundo modelo escolhido foi o Decision Tree.

In [8]:
clf = DecisionTreeClassifier()

clf.fit(trn_features, trn_target)

DecisionTreeClassifier()

### Testando o 2° modelo

In [9]:
error_clf = 1-clf.score(tst_features, tst_target)
print("Erro de classificação: ", error_clf*100, "%")

# Matriz de Confusão
predictions_clf = clf.predict(tst_features)

pd.crosstab(tst_target, predictions_clf, rownames=['Actual Preference'], colnames=['Predicted Preference'])

Erro de classificação:  40.97435897435897 %


Predicted Preference,3,4,5,6,7,8,9
Actual Preference,,,,,,,
3,0,2,4,2,1,0,0
4,1,16,14,14,2,2,0
5,2,24,428,165,40,3,4
6,4,11,184,510,120,22,2
7,0,4,25,85,176,21,0
8,0,0,4,20,15,21,0
9,0,0,0,0,1,1,0


In [10]:
'''
from matplotlib import pyplot as plt
from sklearn import tree
fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(clf, 
                   feature_names=feature_list,  
                   filled=True)
'''

'\nfrom matplotlib import pyplot as plt\nfrom sklearn import tree\nfig = plt.figure(figsize=(25,20))\n_ = tree.plot_tree(clf, \n                   feature_names=feature_list,  \n                   filled=True)\n'

### Referências: <br>
<li> https://blog.softhints.com/pandas-value_counts-multiple-columns/
<li> https://towardsdatascience.com/random-forest-in-python-24d0893d51c0
<li> https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
<li> https://pandas.pydata.org/pandas-docs/stable/index.html
<li> https://chrisalbon.com/machine_learning/trees_and_forests/random_forest_classifier_example/
<li> https://www.python-course.eu/Boosting.php
<li> https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html
<li> https://medium.com/turo-engineering/how-not-to-use-random-forest-265a19a68576
<li> https://stackoverflow.com/questions/38195766/poor-performance-of-randomforestclassifier
<li> https://www.kaggle.com/c/forest-cover-type-prediction/notebooks
<li> https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html
<li> https://www.datacamp.com/community/tutorials/random-forests-classifier-python
<li> https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html#sklearn.ensemble.ExtraTreesClassifier